## Load Dataset

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
# libgomp issue, must import n2 before torch
from n2 import HnswIndex

In [4]:
import sys

sys.path.insert(0, '..')

In [5]:
import os
home_dir = os.getenv('HOME')

https://github.com/anhaidgroup/deepmatcher/blob/master/Datasets.md

In [6]:
import glob
import csv
from tqdm.auto import tqdm

from entity_embed.data_utils.utils import Enumerator

row_dict = {}
id_enumerator = Enumerator()
left_id_set = set()
right_id_set = set()
rows_total = 2554 + 22074
clusters_total = 1154

with tqdm(total=rows_total) as pbar:
    with open(f'{home_dir}/Downloads/walmart_amazon_exp_data/exp_data/tableA.csv') as f:
        for row in csv.DictReader(f):
            row['id'] = id_enumerator[f'left-{int(row["id"])}']
            row['source'] = 'left'
            row_dict[row['id']] = row
            left_id_set.add(row['id'])
            pbar.update(1)
    
    with open(f'{home_dir}/Downloads/walmart_amazon_exp_data/exp_data/tableB.csv') as f:
        for row in csv.DictReader(f):
            row['id'] = id_enumerator[f'right-{int(row["id"])}']
            row['source'] = 'right'
            row_dict[row['id']] = row
            right_id_set.add(row['id'])
            pbar.update(1)

In [7]:
train_true_pair_set = set()
valid_true_pair_set = set()
test_true_pair_set = set()

with open(f'{home_dir}/Downloads/walmart_amazon_exp_data/exp_data/train.csv') as f:
    for row in csv.DictReader(f):
        if int(row['label']) == 1:
            id_left = id_enumerator[f'left-{int(row["ltable_id"])}']
            id_right = id_enumerator[f'right-{int(row["rtable_id"])}']
            train_true_pair_set.add((id_left, id_right))

with open(f'{home_dir}/Downloads/walmart_amazon_exp_data/exp_data/valid.csv') as f:
    for row in csv.DictReader(f):
        if int(row['label']) == 1:
            id_left = id_enumerator[f'left-{int(row["ltable_id"])}']
            id_right = id_enumerator[f'right-{int(row["rtable_id"])}']
            valid_true_pair_set.add((id_left, id_right))

with open(f'{home_dir}/Downloads/walmart_amazon_exp_data/exp_data/test.csv') as f:
    for row in csv.DictReader(f):
        if int(row['label']) == 1:
            id_left = id_enumerator[f'left-{int(row["ltable_id"])}']
            id_right = id_enumerator[f'right-{int(row["rtable_id"])}']
            test_true_pair_set.add((id_left, id_right))
        
display(('train_true_pair_set', len(train_true_pair_set)))
display(('valid_true_pair_set', len(valid_true_pair_set)))
display(('test_true_pair_set', len(test_true_pair_set)))

('train_true_pair_set', 576)

('valid_true_pair_set', 193)

('test_true_pair_set', 193)

## Preprocess

In [8]:
attr_list = ['title', 'category', 'brand', 'modelno', 'price']

In [9]:
import unidecode
import itertools
from entity_embed import default_tokenizer

def clean_str(s):
    s = unidecode.unidecode(s).lower().strip()
    s_tokens = itertools.islice((s_part[:30] for s_part in default_tokenizer(s)), 0, 30)
    return ' '.join(s_tokens)[:300]

for row in tqdm(row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [10]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [11]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [12]:
attr_info_dict = {
    'title': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
        'use_mask': True,
    },
    'title_semantic': {
        'source_attr': 'title',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
    'category': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
        'use_mask': True,
    },
    'category_semantic': {
        'source_attr': 'category',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
    'brand': {
        'field_type': "STRING",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'modelno': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'price': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
}

In [13]:
from entity_embed import AttrInfoDictParser

row_numericalizer = AttrInfoDictParser.from_dict(attr_info_dict, row_dict=row_dict)
row_numericalizer.attr_info_dict

17:06:12 INFO:For attr=title, computing actual max_str_len
17:06:12 INFO:actual_max_str_len=23 must be pair to enable NN pooling. Updating to 24
17:06:12 INFO:For attr=title, using actual_max_str_len=24
17:06:12 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:06:16 INFO:For attr=category, computing actual max_str_len
17:06:16 INFO:actual_max_str_len=15 must be pair to enable NN pooling. Updating to 16
17:06:16 INFO:For attr=category, using actual_max_str_len=16
17:06:16 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:06:19 INFO:For attr=brand, computing actual max_str_len
17:06:19 INFO:For attr=brand, using actual_max_str_len=48
17:06:19 INFO:For attr=modelno, computing actual max_str_len
17:06:19 INFO:For attr=modelno, using actual_max_str_len=48
17:06:19 INFO:For attr=price, computing actual max_str_len
17:06:19 INFO:For attr=price, using actual_max_str_len=10


{'title': NumericalizeInfo(attr='title', field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=24, vocab=None, n_channels=8, embed_dropout_p=0.2, use_attention=True, use_mask=True),
 'title_semantic': NumericalizeInfo(attr='title', field_type=<FieldType.SEMANTIC_MULTITOKEN: 'semantic_multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"',

In [14]:
from entity_embed import PairwiseDataModule

datamodule = PairwiseDataModule(
    row_dict=row_dict,
    row_numericalizer=row_numericalizer,
    batch_size=10,
    row_batch_size=16,
    train_true_pair_set=train_true_pair_set,
    valid_true_pair_set=valid_true_pair_set,
    test_true_pair_set=test_true_pair_set,
    random_seed=random_seed
)

## Training

In [15]:
from entity_embed import LinkageEmbed

ann_k = 100
model = LinkageEmbed(
    datamodule,
    ann_k=ann_k,
    embedding_size=300
)

In [16]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 100
early_stop_callback = EarlyStopping(
   monitor='valid_recall_at_0.3',
   min_delta=0.00,
   patience=20,
   verbose=True,
   mode='max'
)
tb_log_dir = 'tb_logs'
tb_name = 'walmart-amazon'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_log_dir, name=tb_name)
)

17:06:19 INFO:GPU available: True, used: True
17:06:19 INFO:TPU available: None, using: 0 TPU cores
17:06:19 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [17]:
trainer.fit(model, datamodule)

17:06:21 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 22.6 M
1 | losser      | SupConLoss | 0     
-------------------------------------------
13.7 M    Trainable params
8.9 M     Non-trainable params
22.6 M    Total params


17:06:22 WARNING:true_pair_set has 38 less elements than transitive_true_pair_set. This means there are transitive true pairs not included in true_pair_set.


1

In [18]:
model.blocker_net.get_signature_weights()

{'title': 0.2670438587665558,
 'title_semantic': 0.23384031653404236,
 'category': 0.07413995265960693,
 'category_semantic': 0.0520101822912693,
 'brand': 0.1775631308555603,
 'modelno': 0.13716518878936768,
 'price': 0.05823740363121033}

In [19]:
from entity_embed import validate_best

validate_best(trainer)

{'valid_f1_at_0.3': 0.37954768928220256,
 'valid_f1_at_0.5': 0.7020109689213895,
 'valid_f1_at_0.7': 0.8922305764411028,
 'valid_f1_at_0.9': 0.6120996441281138,
 'valid_pair_entity_ratio_at_0.3': 2.174142480211082,
 'valid_pair_entity_ratio_at_0.5': 0.9340369393139841,
 'valid_pair_entity_ratio_at_0.7': 0.5435356200527705,
 'valid_pair_entity_ratio_at_0.9': 0.23218997361477572,
 'valid_precision_at_0.3': 0.2342233009708738,
 'valid_precision_at_0.5': 0.5423728813559322,
 'valid_precision_at_0.7': 0.8640776699029126,
 'valid_precision_at_0.9': 0.9772727272727273,
 'valid_recall_at_0.3': 1.0,
 'valid_recall_at_0.5': 0.9948186528497409,
 'valid_recall_at_0.7': 0.9222797927461139,
 'valid_recall_at_0.9': 0.44559585492227977}

## Testing

In [20]:
trainer.test(ckpt_path='best', verbose=False)

[{'test_f1_at_0.3': 0.4341957255343082,
  'test_f1_at_0.5': 0.8286334056399132,
  'test_f1_at_0.7': 0.9048843187660668,
  'test_f1_at_0.9': 0.5912408759124088,
  'test_pair_entity_ratio_at_0.3': 1.8125,
  'test_pair_entity_ratio_at_0.5': 0.6979166666666666,
  'test_pair_entity_ratio_at_0.7': 0.5104166666666666,
  'test_pair_entity_ratio_at_0.9': 0.2109375,
  'test_precision_at_0.3': 0.27729885057471265,
  'test_precision_at_0.5': 0.7126865671641791,
  'test_precision_at_0.7': 0.8979591836734694,
  'test_precision_at_0.9': 1.0,
  'test_recall_at_0.3': 1.0,
  'test_recall_at_0.5': 0.9896373056994818,
  'test_recall_at_0.7': 0.9119170984455959,
  'test_recall_at_0.9': 0.41968911917098445}]

## Testing manually 

In [21]:
from entity_embed.data_utils.utils import id_pairs_to_cluster_mapping_and_dict, cluster_dict_to_filtered_row_dict

__, test_cluster_dict = id_pairs_to_cluster_mapping_and_dict(test_true_pair_set)
test_row_dict = cluster_dict_to_filtered_row_dict(row_dict, test_cluster_dict)
test_left_vector_dict, test_right_vector_dict = model.predict(
    row_dict=test_row_dict,
    left_id_set=left_id_set,
    right_id_set=right_id_set,
    batch_size=16
)

In [22]:
embedding_size = model.blocker_net.embedding_size

In [23]:
assert (len(test_left_vector_dict) + len(test_right_vector_dict)) == len(test_row_dict)

In [24]:
%%time

from entity_embed import ANNLinkageIndex

ann_index = ANNLinkageIndex(embedding_size=embedding_size)
ann_index.insert_vector_dict(left_vector_dict=test_left_vector_dict, right_vector_dict=test_right_vector_dict)
ann_index.build()

CPU times: user 628 ms, sys: 6.59 ms, total: 635 ms
Wall time: 140 ms


In [25]:
%%time

sim_threshold = 0.3
found_pair_set = ann_index.search_pairs(
    k=ann_k,
    sim_threshold=sim_threshold,
    left_vector_dict=test_left_vector_dict,
    right_vector_dict=test_right_vector_dict,
)

CPU times: user 480 ms, sys: 0 ns, total: 480 ms
Wall time: 69.7 ms


In [26]:
from entity_embed.evaluation import pair_entity_ratio

pair_entity_ratio(len(found_pair_set), len(test_row_dict))

1.8125

In [27]:
from entity_embed.evaluation import precision_and_recall, f1_score

precision, recall = precision_and_recall(found_pair_set, test_true_pair_set)
precision, recall

(0.27729885057471265, 1.0)

In [28]:
false_positives = list(found_pair_set - test_true_pair_set)
len(false_positives)

503

In [29]:
false_negatives = list(test_true_pair_set - found_pair_set)
len(false_negatives)

0

In [30]:
cos_similarity = lambda a, b: np.dot(a, b)

In [31]:
for (id_left, id_right) in false_negatives[:10]:
    display(
        (
            cos_similarity(test_left_vector_dict[id_left], test_right_vector_dict[id_right]),
            row_dict[id_left], row_dict[id_right]
        )
    )